In [1]:
from nltk.tokenize import sent_tokenize
from collections import Counter
import string

In [2]:
with open('alice_wonderland.txt', 'r', encoding='utf-8') as fh:
    alltext = fh.read()
cleaned_text = alltext.lower()

In [3]:
cleaned_text = cleaned_text.translate({ord('\n'):' ', 
                                  ord('\"'):'', 
                                  ord('’'):'',
                                  ord('‘'):'',    
                                  ord('\''):''
                                 })

In [4]:
sents = sent_tokenize(cleaned_text)

In [5]:
# now that we have obtained sentences, apply "cleaning" transformations: 
# 1. remove punctuations
# 2. add sentence start and end markers
translator = str.maketrans('', '', string.punctuation)
sents = [x for x in map(lambda x: x.translate(translator), sents)]
sents = [x for x in map(lambda x: x + ' </s>', sents)]


In [6]:
all_tokens = []
for sent in sents:
    thiswords = [x for x in filter(lambda x: len(x)>0, sent.split(' '))]
    all_tokens.extend(thiswords)

all_types = set(all_tokens)
len(all_types)

2834

In [7]:
bigrams = [(all_tokens[i], all_tokens[i+1]) for i in range(len(all_tokens)-1)]
count = Counter(bigrams)
count.most_common(20)

[(('said', 'the'), 209),
 (('of', 'the'), 130),
 (('</s>', 'i'), 117),
 (('</s>', 'the'), 116),
 (('said', 'alice'), 115),
 (('</s>', 'said'), 109),
 (('in', 'a'), 97),
 (('</s>', 'alice'), 81),
 (('and', 'the'), 80),
 (('in', 'the'), 78),
 (('it', 'was'), 74),
 (('to', 'the'), 69),
 (('</s>', 'and'), 68),
 (('the', 'queen'), 65),
 (('as', 'she'), 61),
 (('the', 'king'), 60),
 (('at', 'the'), 60),
 (('she', 'had'), 60),
 (('alice', '</s>'), 59),
 (('a', 'little'), 59)]